In [12]:
import vk_api
import sqlite3

In [13]:
access_token = "vk1.a.iB5g19B_HV1O4Vl5mkph4mMTBvWHtQcFgbuAkuUIfbtN9fQ6J5tapohBqMQaMrimB7sngf28DeObk8FFBZ_SDkqB_cUPjVYqN-i0q2zpPWNm2Q1XQGn1zup80BQXb6MxE35gqTghuhS8xdILf036_vUQLJzIo9rF73QxNASS4zvNcElkCIe_EpXwgq5ZkaVuW5s6fcHqQfpGY5WtvtA3qg"

In [14]:
def get_group_info(vk_session, group_id):
    try:
        # Получение информации о группе
        group_info = vk_session.method('groups.getById', {'group_id': group_id, 'fields': 'description,activity'})
        if group_info:
            group_info = group_info[0]
            name = group_info.get('name')
            about = group_info.get('description')
            activity = group_info.get('activity')

            return name, about, activity
        else:
            return None
    except Exception as e:
        print(f"Ошибка при получении информации о группе: {e}")
        return None

In [15]:
def get_last_posts(vk_session, group_id, count=5):
    try:
        # Получение последних пяти постов
        posts = vk_session.method('wall.get', {'owner_id': -group_id, 'count': count})
        return [post['text'] for post in posts['items']]
    except Exception as e:
        print(f"Ошибка при получении постов: {e}")
        return []

In [27]:
def insert_user_data(db_path, group_id, name, description, theme, texts):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO groups_info (group_id, name, group_theme, about, post_texts) VALUES (?, ?, ?, ?, ?)", (group_id, name, theme, description, texts))
    conn.commit()
    conn.close()

In [28]:
def get_unique_group_ids(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT group_ids FROM users")

    group_ids = set()
    for row in cursor.fetchall():
        ids = row[0].split(',')
        group_ids.update(ids)

    conn.close()
    return list(group_ids)

In [32]:
def main():
    db_users = "../../data/users.db"
    vk_session = vk_api.VkApi(token=access_token)

    unique_group_ids = get_unique_group_ids(db_users)
    for group_id in unique_group_ids:
        # Проверяем, что group_id действительно число
        if not group_id.isdigit():
            continue

        group_info = get_group_info(vk_session, int(group_id))
        if group_info:
            posts = get_last_posts(vk_session, int(group_id))
            post_texts = ','.join(posts)  # Объединение текстов постов через запятую

            # Здесь предполагается, что функция insert_user_data уже определена
            # и правильно вставляет данные в базу данных.
            insert_user_data(db_users, int(group_id), group_info[0], group_info[2], group_info[1], post_texts)


In [33]:
if __name__ == '__main__':
    main()

Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: this wall available only for community members
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: this wall available only for community members
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: this wall available only for community members
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: this wall available only for community members
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении постов: [15] Access denied: group is blocked
Ошибка при получении п

KeyboardInterrupt: 